In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


11/7 評価関数F2を各手法ごとに追記(上書きでミスってた時が怖いため，ファイル名を"データセット-手法-addF2.csv"に変更)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
#sysはコマンドライン引数
import sys
import csv
import math
c = 3

In [3]:
#fcm,fecbo-dbクラスタ中心計算部分関数作成
def v_calculation(u_m,c,dimension,X):
    #分母計算(クラスタサイズになるはず)
    v_denominator = np.sum(u_m,axis=0)
    #分子計算
    v_numerator = np.zeros((c,dimension))
    for j in range(c):
        for i in range(X_size):
            v_numerator[j]+= u_m[i][j]*X[i]
   
    v = np.zeros((c,dimension ))
    for i in range(c):
        v[i] = v_numerator[i]/v_denominator[i]
    return v

In [17]:
#引数(クラスタ数(c),クラスター(cluster),データセット(X),クラスター中心(v)、\
#ファジィ化パラム(param(mかlam_1))、帰属度(u))

def calc_F(c,cluster,X,v,param,u,model):
    #中心間距離の作成
    center_distance = np.zeros((c,c))
    for i in range(c):
        for j in range(c):
            center_distance[i][j]= np.sum((v[i]-v[j])**2)
    print("center_distance")
    print(center_distance)
    #dにはd11,d12,d13,d14,d21,d22,,,dn1,,dncの順でリスト作る
    d_list=[]
    for i in range(X_size):
        for j in range(c):
            d_list.append(np.sum((X[i]-v[j])**2))
    d = np.array(d_list)
    d = d.reshape([X_size,c])
    print("d[0]",d[0])

    new_d = np.zeros((X_size,c))
    #ここが怪しい
    for i in range(c):
        new_d[cluster == i] = d[cluster == i]+center_distance[i]
    print("new_d[0] = d_{ki}+d_{ii*}",new_d[0])

    #評価関数F_1計算(ついでにJも再計算してあってるか確認)
    F_1 = 0
    for i in range(X_size):
        F_1 += np.sum(u[i] * new_d[i])
    #目的関数の再計算（確認）
    kakunin_J = 0
    if (model =="fecbo-db") or (model == "fcm"):
        u_m = u**param
        for i in range(X_size):
            kakunin_J += np.sum(u_m[i] * d[i])
    if model =="fecbo-e":
        for i in range(X_size):
            for j in range(c):
                kakunin_J += u[i][j]*d[i][j]+param*u[i][j]*math.log(u[i][j])
    
        
    #評価関数F_2計算
    F_2 = 0
    new_u = np.copy(u)
    for i in range(X_size):
        new_u[i][cluster[i]] = 0
    for i in range(X_size):
        F_2 += np.sum(new_u[i] * new_d[i])
    #この初期値のときの結果が優れている場合，bestを更新
    print("Jの値:",J)
    print("Jの値を再計算:",kakunin_J)
    print("F_1の値:",F_1)
    print("F_2の値:",F_2)
    return F_1,F_2,d,new_d

## FCMへのF2の記入

In [5]:
df = pd.read_csv("./iris_csvFile/Iris-fcm.csv")
u = df.loc[:,"u1":"u3"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
m = 3
u_m = u**m
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F,F2
0,5.1,3.5,1.4,0.2,0,0.915676,0.034733,0.049591,29.110238,834.025086,786.268279
1,4.9,3.0,1.4,0.2,0,0.827251,0.070570,0.102178,29.110238,834.025086,786.268279
2,4.7,3.2,1.3,0.2,0,0.835861,0.067695,0.096444,29.110238,834.025086,786.268279
3,4.6,3.1,1.5,0.2,0,0.801531,0.081068,0.117401,29.110238,834.025086,786.268279
4,5.0,3.6,1.4,0.2,0,0.894932,0.043400,0.061669,29.110238,834.025086,786.268279
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1,0.069029,0.715875,0.215096,29.110238,834.025086,786.268279
146,6.3,2.5,5.0,1.9,1,0.097676,0.466216,0.436108,29.110238,834.025086,786.268279
147,6.5,3.0,5.2,2.0,1,0.064641,0.698666,0.236693,29.110238,834.025086,786.268279
148,6.2,3.4,5.4,2.3,1,0.093683,0.624896,0.281420,29.110238,834.025086,786.268279


In [18]:
v = v_calculation(u_m,c,dimension,X)
v

array([[5.00106653, 3.38935647, 1.49425962, 0.25194824],
       [6.69509539, 3.03751182, 5.55144409, 2.03544793],
       [5.90997311, 2.79144758, 4.37839845, 1.39638155]])

In [19]:
F_1,F_2,d,new_d = calc_F(c,cluster,X,v,m,u,"fcm")

center_distance
[[ 0.         22.63514541 10.81159058]
 [22.63514541  0.          2.46140653]
 [10.81159058  2.46140653  0.        ]]
d[0] [ 0.03361332 23.36158179 11.46028911]
new_d[0] = d_{ki}+d_{ii*} [3.36133186e-02 4.59967272e+01 2.22718797e+01]
Jの値: 29.11023838969788
Jの値を再計算: 29.11023838969677
F_1の値: 834.0250861732662
F_2の値: 786.2682785314418


In [29]:
u[0]

array([0.91567603, 0.03473333, 0.04959065])

In [31]:
np.sum(u[0]*new_d[0])

2.7328752672861913

In [27]:
u[0]*new_d[0]

array([0.03077891, 1.59761946, 1.1044769 ])

In [32]:
u[57]

array([0.32357776, 0.22702545, 0.44939679])

In [33]:
d[57]

array([4.80932041, 9.7699417 , 2.49333844])

In [34]:
new_d[57]

array([15.62091099, 12.23134823,  2.49333844])

In [38]:
u[57]*d[57]

array([1.55618911, 2.21802543, 1.12049829])

In [35]:
u[57]*new_d[57]

array([5.05457934, 2.77682736, 1.12049829])

In [39]:
np.sum(u[57]*new_d[57])

8.951904992927656